Read in a spreadsheet of data from sam 

In [1]:
import os,sys,csv,string
from csv import reader

# input is in sys.argv[1]

# if len(sys.argv) < 2:
#     print ("Need to enter filename for csv")
#     sys.exit(0)

# filename = sys.argv[1]
filename = "SamEvents.csv"
    
if not "csv" in filename:
  print (" need to enter a csv filename")
  sys.exit(1)

file = open(filename,'r')
data = []

for line in reader(file,dialect='excel'):
  data.append(line)

headers = data[0]

# pull out records for each process ID and parse

info = {}
for line in data[1:]:

    mydict = {}
    timestamp = line[0]
    
    # clean up ID #'s'
    for field in range(0,len(headers)):
        entry = line[field]
        if headers[field] in ["process_id","file_id","file_size","kafka.timestamp"] and len(entry)>0:
            entry  = int(entry.replace(",",""))
        
        if headers[field] == "file_size" and entry == "":
            entry = 0
        
        mydict[headers[field]] = entry
    
    process = mydict["process_id"] 


    if not process in info:
        info[process] = {}
    info[process][timestamp] = mydict
    
# now merge records for each jobid into a single record

single = []

keys = None
for a  in info:
    process = info[a]
    
    stamps = list(process.keys())
    
    file_id = ""
    file_url = ""
    ended  = False
    file_size = 0
    kafka_begin = 0
    
    # need to sort them by time and then pick out more recent data
    
    file_state = "Unknown"
    process_state = "Unknown"
    for i in sorted(stamps):
        # info from all records
        lasttime = i
        record = process[i]
        timestamp = i
        event = record["event"]
        
        if record["file_state"]!= "":
            file_state = record["file_state"]
        if record["process_state"]!= "":
            process_state = record["process_state"]
        process_state = record["process_state"]
        if record["file_size"] != 0:
            file_size =record["file_size"]
        #info just from the transfers
        if record["file_state"] not in ["transferred","consumed"] and record["process_state"] not in ["completed","finished"]:
            continue
        if record["file_id"] != "":
            file_id =record["file_id"]
        if kafka_begin == 0:
            kafka_begin = record["kafka.timestamp"]
        kafka_end = record["kafka.timestamp"]
            
        if record["file_url"] != "":
            file_url = record["file_url"]
        if record["node"] != "":
            node = record["node"]
        if file_state == "transferred":
            start_time = timestamp
        if file_state == "consumed" or process_state in ["completed","finished"] :
            end_time = timestamp
            ended = True
    if not ended: 
        end_time = lasttime
    # make the records out of the bits you collected
    new = {}
    new["job_id"]=a
    new["start"] = start_time
    new["end"]= end_time
    new["node"]=node
    new["file_id"]=file_id
    new["file_url"]=file_url
    new["file_size"]=file_size
    new["last_file_state"]=file_state
    new["last_process_state"]=process_state
    new["ended"]=ended
    new["kafka_begin"]= kafka_begin
    new["kafka_end"]= kafka_end
    new["kafka_alivetime"]=(kafka_end-kafka_begin)/1000.
    if file_size > 0:
        single.append(new)
   
    
keys = single[0].keys()        
        
# now dump to a csv file
csv_file = "SamTransfers.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=keys)
        writer.writeheader()
        
        for data in single:
            
            writer.writerow(data)
            
except IOError:
    print("I/O error")
    
    
